# Prepping the data

In [1]:
import pandas as pd

In [2]:
athletes = pd.read_csv("athlete_events.csv")
athletes.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [4]:
# only the summer olympics
athletes = athletes[athletes["Season"] == "Summer"]
athletes.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
26,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932 Summer,1932,Summer,Los Angeles,Athletics,Athletics Women's 100 metres,NaN


In [5]:
def team_summary(data):
    return pd.Series({
        'team': data.iloc[0,:]["NOC"],
        'country': data.iloc[-1,:]["Team"],
        'year': data.iloc[0,:]["Year"],
        'events': len(data['Event'].unique()),
        'athletes': data.shape[0],
        'age': data["Age"].mean(),
        'height': data['Height'].mean(),
        'weight': data['Weight'].mean(),
        'medals': sum(~pd.isnull(data["Medal"]))
    })

team = athletes.groupby(["NOC", "Year"]).apply(team_summary)
team.head()

team      country  year  events  athletes        age    height  \
NOC Year                                                                  
AFG 1936  AFG  Afghanistan  1936       4        16  24.266667       NaN   
    1948  AFG  Afghanistan  1948       2        25        NaN       NaN   
    1956  AFG  Afghanistan  1956       1        12        NaN       NaN   
    1960  AFG  Afghanistan  1960      13        16  23.312500  170.6875   
    1964  AFG  Afghanistan  1964       8         8  22.000000  161.0000   

           weight  medals  
NOC Year                   
AFG 1936      NaN       0  
    1948      NaN       0  
    1956      NaN       0  
    1960  69.4375       0  
    1964  64.2500       0

In [7]:
team = team.reset_index(drop=True)
team = team.dropna()
team

,team,country,year,events,athletes,age,height,weight,medals
0,AFG,Afghanistan,1960,13,16,23.312500,170.687500,69.437500,0
1,AFG,Afghanistan,1964,8,8,22.000000,161.000000,64.250000,0
2,AFG,Afghanistan,1968,5,5,23.200000,170.200000,70.000000,0
3,AFG,Afghanistan,1972,8,8,29.000000,168.333333,63.750000,0
4,AFG,Afghanistan,1980,11,11,23.636364,168.363636,63.181818,0
...,...,...,...,...,...,...,...,...,...
2608,ZIM,Zimbabwe,2000,19,26,24.961538,178.960000,71.080000,0
2609,ZIM,Zimbabwe,2004,11,14,25.071429,177.785714,70.500000,3
2610,ZIM,Zimbabwe,2008,15,16,26.062500,171.928571,63.714286,4
2611,ZIM,Zimbabwe,2012,8,9,27.333333,174.444444,65.222222,0


In [9]:
def prev_medals(data):
    data = data.sort_values("year", ascending=True)
    data["prev_medals"] = data["medals"].shift(1)
    data["prev_3_medals"] = data.rolling(3, closed="left", min_periods=1).mean()["medals"]
    return data

team = team.groupby(["team"],group_keys=True).apply(prev_medals)
team = team.reset_index(drop=True)
team = team[team["year"] > 1960]
team = team.round(1)

C:\Users\pgyot\AppData\Local\Temp\ipykernel_40640\1744429881.py:4: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['country', 'team'], dtype='object')
  data["prev_3_medals"] = data.rolling(3, closed="left", min_periods=1).mean()["medals"]


In [10]:
team.to_csv("teams.csv", index=False)